In [2]:
import requests
from bs4 import BeautifulSoup

In [3]:
r = requests.get('https://www.trustpilot.com')
print(r.status_code)
print(r.status_code == requests.codes.ok)
print(requests.codes['temporary_redirect'])
print(requests.codes.teapot)
print(requests.codes['o/'])

200
True
307
418
None


- review card: DONE
- Username: DONE
- Rating: DONE 
- Title: DONE
- Content: DONE
- date:MISSING 
- Useful score: SKIP
- Verified/Invited: MISSING
- Number of reviews wrote by customer: DONE
- Reply: DONE

In [4]:
# review_card = soup.find_all('div', class_='review-card')
# first_review = review_card[0]
    
# first_date = first_review.find('div', class_='review-content-header__dates')
# first_date = first_review.find('time')
# first_date

# first_verified = first_review.find('div', class_='review-label-tooltip-trigger')
# print(first_verified)
# first_verified = first_review.find('div', class_='review-label-tooltip-trigger')
# if first_verified:
#     if first_verified.get_text(strip=True) == 'Invited':
#         verified_status = 'Invited'
#     elif first_verified.get_text(strip=True) == 'Reviewer contacted':
#         verified_status = 'Reviewer contacted'
#     else:
#         verified_status = 'other'
# else:
#     verified_status = None
# print(verified_status)

In [43]:
names = []
ratings = []
titles = []
contents = []
rev_wrote = []
replies = []
company_names = []
num_reviews = []
company_ratings = []
websites = []

PAGES = 5
COMPANY ='www.monday.com'

for p in range(1, PAGES):
    page_url = requests.get('https://www.trustpilot.com/review/' + COMPANY + '?page=' + str(p))
    soup = BeautifulSoup(page_url.content, 'html.parser')
    if p == 1:
        company_name = soup.find('span', class_='multi-size-header__big').get_text(strip=True)
        company_names.append(company_name)
        num_review = soup.find('h2', class_='header--inline').get_text(strip=True)
        num_review = ''.join(filter(str.isdigit, num_review))
        num_reviews.append(num_review)
        company_rating = soup.find('p', class_='header_trustscore').get_text()
        company_ratings.append(company_rating)
        website_link = soup.find('a', {'class': 'badge-card__section badge-card__section--hoverable company_website'})
        website = website_link.get('title')
        websites.append(website)
                
    review_card = soup.find_all('div', class_='review-card')
    for review in review_card:
        name = review.find('div', class_='consumer-information__name').get_text(strip=True)
        names.append(name)
        rating = review.find('img').attrs.get('alt')
        ratings.append(rating)
        title = review.find('a', class_='link link--large link--dark').get_text()
        titles.append(title)
        if review.find('p', class_='review-content__text'):
            content = review.find('p', class_='review-content__text').get_text(strip=True)
            
        else:
            content = None
        contents.append(content)
        rev_written = review.span.get_text()
        rev_wrote.append(rev_written)
        reply = review.find('div', class_='review__company-reply')
        if reply:
            replies.append(True)
        else:
            replies.append(False)
    

In [45]:
company_names = []
num_reviews = []
company_ratings = []
websites = []

PAGES = 5
COMPANY ='www.monday.com'

for p in range(1, PAGES):
    page_url = requests.get('https://www.trustpilot.com/review/' + COMPANY + '?page=' + str(p))
    soup = BeautifulSoup(page_url.content, 'html.parser')
    if p == 1:
        company_name = soup.find('span', class_='multi-size-header__big').get_text(strip=True)
        company_names.append(company_name)
        num_review = soup.find('h2', class_='header--inline').get_text(strip=True)
        num_review = ''.join(filter(str.isdigit, num_review))
        num_reviews.append(num_review)
        company_rating = soup.find('p', class_='header_trustscore').get_text()
        company_ratings.append(company_rating)
        website_link = soup.find('a', {'class': 'badge-card__section badge-card__section--hoverable company_website'})
        website = website_link.get('title')
        websites.append(website)

In [8]:
import pandas as pd
test_df = pd.DataFrame({'names': names,
                       'ratings': ratings,
                       'titles': titles,
                       'contents': contents,
                       'rev_wrote': rev_wrote,
                       'replies': replies})
display(test_df.info())
display(test_df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380 entries, 0 to 379
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   names      380 non-null    object
 1   ratings    380 non-null    object
 2   titles     380 non-null    object
 3   contents   366 non-null    object
 4   rev_wrote  380 non-null    object
 5   replies    380 non-null    bool  
dtypes: bool(1), object(5)
memory usage: 15.3+ KB


None

,names,ratings,titles,contents,rev_wrote,replies
0,Joel Vargas,3 stars: Average,\n Decently useful site\n ...,"Decently useful site, but it’s the bane of my ...",12 reviews,False
1,Nita Kusumadewi,5 stars: Excellent,\n Excellent assistance to ...,Excellent assistance to make my life easier to...,1 review,False
2,Susan Allen,1 star: Bad,\n Dishonest company that h...,An invitation to join Monday.com was sent to m...,2 reviews,False
3,Rain Master,1 star: Bad,\n crazy expensive\n ...,"crazy expensive, especially for team in Asia, ...",1 review,True
4,Igaln,2 stars: Poor,\n Too expensive too quickl...,Same reason as users below; $1200/year additi...,1 review,True


In [46]:
df1 = pd.DataFrame({'company_names': company_names,
                   'num_reviews': num_reviews,
                   'company_ratings': company_ratings,
                   'websites': websites})
display(df1.head())

,company_names,num_reviews,company_ratings,websites
0,monday.com,2647,4.6,www.monday.com


In [346]:
test_df.to_csv('reviews.csv')

In [48]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


def check_status():
    """Checks status of webpage"""
    r = requests.get('https://www.trustpilot.com')
    print(r.status_code)
    print(r.status_code == requests.codes.ok)
    print(requests.codes['temporary_redirect'])
    print(requests.codes.teapot)
    print(requests.codes['o/'])


def scrap():
    """Scrap a company's reviews from their TrustPilot page."""
    names = []
    ratings = []
    titles = []
    contents = []
    rev_wrote = []
    replies = []
    website = []
    urls = []
    PAGES = 2
    COMPANY = 'www.monday.com'



    for p in range(1, PAGES):

        page_url = requests.get('https://www.trustpilot.com/review/' + COMPANY + '?page=' + str(p))
        soup = BeautifulSoup(page_url.content, 'html.parser')
        review_card = soup.find_all('div', class_='review-card')

        # find website of the company
        # I do it just one time
        #TODO LOOK AT THIS we need to put
        if p == 1:
            web_tag = soup.find_all('a', class_="badge-card__section badge-card__section--hoverable company_website")
            for a in web_tag:
                website.append(a['href'])

        # get url for each user
        user_url = soup.find_all('a', href=True)
        for a in user_url:
            user_id = a['href']
            if '/users/5' in user_id and user_id not in urls:
                urls.append(user_id)
        for review in review_card:

            # Username
            name = review.find('div', class_='consumer-information__name').get_text(strip=True)
            names.append(name)
            # Rating
            rating = review.find('img').attrs.get('alt')
            ratings.append(rating)
            # Review title
            title = review.find('a', class_='link link--large link--dark').get_text()
            titles.append(title)
            # Review content
            if review.find('p', class_='review-content__text'):
                content = review.find('p', class_='review-content__text').get_text(strip=True)
            else:
                content = None
            contents.append(content)
            # Number of reviews wrote by user
            rev_written = review.span.get_text()
            rev_wrote.append(rev_written)
            # Replied received
            reply = review.find('div', class_='review__company-reply')
            if reply:
                replies.append(True)
            else:
                replies.append(False)

            # country and parse another page
    countries = parse_another_page(urls)

    reviews_dict = {'names': names,
                    'ratings': ratings,
                    'titles': titles,
                    'contents': contents,
                    'rev_wrote': rev_wrote,
                    'replies': replies,
                    'countries': countries
                    }

    return reviews_dict


def parse_another_page(urls):
    lst = []
    for url in urls:
        page_url = requests.get('https://www.trustpilot.com/' + url)
        soup = BeautifulSoup(page_url.content, 'html.parser')
        countries = soup.find('div', class_='user-summary-location')
        if countries is not None:
            lst.append(countries.text.strip().strip('\n'))
    return lst


def export_csv():
    """Stores results to pandas df and creates a csv file."""
    rev_dict = scrap()
    reviews_df = pd.DataFrame(rev_dict)
    reviews_df.to_csv('reviews2.csv')


def main():
    # print(scrap())
    export_csv()


if __name__ == '__main__':
    main()